# Clasificación con Redes Neuronales (y Tensorflow)



## 1. Arquitectura típica de una red neuronal para clasificación

Las partes comunes a todas las redes neuronales profundas son las siguientes:
* Capa de entrada
* Algunas capas ocultas
* Capa de salida

Para el uso de redes neuronales para clasificación, es usual utilizar los siguientes elementos típicos.

| **Elemento** | **Clasificación binaria** | **Clasificación multiclase** |
| --- | --- | --- |
| Capa de entrada  | Igual tamaño que el número de atributos | Similar a clasificación binaria |
| Capas ocultas | Mínimo 1, máximo ilimitado | Similar a clasificación binaria |
| Neuronas por capa oculta | Generalmente de 10 a 100 | Similar a clasificación binaria |
| Capa de salida | Tamaño 1 (una clase o la otra) | 1 por clase |

Algunos otros elementos son los siguientes:

| **Elemento** | **Clasificación binaria** | **Clasificación multiclase** |
| --- | --- | --- |
| Activación oculta | Usualmente ReLU (rectified linear unit) | Similar a clasificación binaria |
| Activación de salida | Sigmoidea | Softmax |
| Función de pérdida  | Entropía cruzada binaria (Cross entropy) | Entropía cruzada categórica |
| Optimizador | SGD (stochastic gradient descent), Adamk, etc. | Similar a clasificación binaria |

In [ ]:
import pandas as pd
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt

import tensorflow as tf

print(tf.__version__)

## 2. Creación de datos 

Por facilidad se utilizará la función `make_circles()` de Scikit-Learn.



In [ ]:
# Número de muestras
n_samples = 1000
# Crear círculos
X, y = make_circles(n_samples, noise = 0.03, random_state = 42)

# Atributos
print(X)

In [ ]:
# Mostrar las 10 primeras etiquetas
print(y[:10])

In [ ]:
# Crear un dataframe de atributos y etiquetas
circulos = pd.DataFrame({"X0":X[:, 0], "X1":X[:, 1], "etiqueta":y})
circulos.head()

In [ ]:
# Verificar los valores de las etiquetas
circulos.etiqueta.value_counts()

Se tiene un problema de clasificación binaria ya que solo se tiene 2 etiquetas: 1 y 0

In [ ]:
# Visualización de los datos
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu);

Sería conveniente utilizar el [TensorFlow Playground](https://playground.tensorflow.org/#activation=relu&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=2,2&seed=0.93799&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false&regularization_hide=true&regularizationRate_hide=true&batchSize_hide=true) para ver experimentalmente el efecto de los componentes de una red neuronal. Intentar ajustar los diferentes hiperparámetros que se ve y hacer click a play para ver el entrenamiento de una red neuronal.


**Tamaños de las entradas y salidas**

In [ ]:
# Verificar los tamaños de los atributos y de las etiquetas
print(X.shape)
print(y.shape)

In [ ]:
# Primera instancia de atributos y de etiquetas
print(X[0])
print(y[0])

## 3. Modelamiento

1. Crear el modelo
2. Compilar el modelo
3. Ajustar (entrenar) el modelo

In [ ]:
# Semilla aleatoria
tf.random.set_seed(42)

# 1. Creación del modelo
modelo_1 = tf.keras.Sequential([
                                tf.keras.layers.Dense(1)
])

# 2. Compilar el modelo
modelo_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),     # Solo 2 clases
                optimizer = tf.keras.optimizers.SGD(),
                metrics = ['accuracy'])

# 3. Entrenamiento del modelo
modelo_1.fit(X, y, epochs=5)

In [ ]:
# Entrenamiento usando más épocas
modelo_1.fit(X, y, epochs=200, verbose=0) 

In [ ]:
modelo_1.evaluate(X, y)

**Mejoras**: Se puede añadir más capas

In [ ]:
tf.random.set_seed(42)

modelo_2 = tf.keras.Sequential([
                                tf.keras.layers.Dense(1), 
                                tf.keras.layers.Dense(1) 
])

modelo_2.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.SGD(),
                 metrics = ['accuracy'])

historia = modelo_2.fit(X, y, epochs=100, verbose=0)

In [ ]:
# Evaluate the model
modelo_2.evaluate(X, y)

## 4. Mejora del modelo

Se puede realizar mejoras de manera similar a como se realiza en regresión.

In [ ]:
tf.random.set_seed(42)

modelo_3 = tf.keras.Sequential([
                               tf.keras.layers.Dense(100), # añadir 100 neuronas
                               tf.keras.layers.Dense(10),  # capa con 10 neuronas
                               tf.keras.layers.Dense(1)
])

modelo_3.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),  # Uso de Adam
                metrics = ['accuracy'])

modelo_3.fit(X, y, epochs=100, verbose=0)   # Usar 100 pasadas por las datas

In [ ]:
# Evaluar el modelo
modelo_3.evaluate(X, y)

## 5. Visualización 

Cuando un modelo se comporta de manera extraña o hay algo que no parece correcto, lo usual es realizar una visualización para inspeccionar el modelo e inspeccionar las predicciones del modelo. 

Para visualizar, en este caso, se va a realizar una función `plot_decision_boundary()` que realizará lo siguiente:
* Tomar los atributos (X) y etiquetas como entrada (y)
* Crear una malla (meshgrid) de los valores de X
* Graficar las predicciones y la línea entre las diferentes zonas (donde se encuentra cada clase única)

In [ ]:
import numpy as np

def plot_decision_boundary(model, X, y):
  """
  Graficar la frontera de decisión 
  """
  # Definir los ejes de las fronteras y crear la malla
  x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
  y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
  xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                       np.linspace(y_min, y_max, 100))
  
  # Crear los valores de X (se va a predecir para esos valores)
  x_in = np.c_[xx.ravel(), yy.ravel()]
  # Realizar las predicciones usando el modelo entrenado
  y_pred = model.predict(x_in)

  # Verificar si es multi clase
  if model.output_shape[-1] > 1: # Verificar la dimensión final de la salida: si es > 1, es multi clase
    print("Realizando clasificación multiclase ...")
    # Se tiene que modificar los tamaños de las predicciones para graficarlas
    y_pred = np.argmax(y_pred, axis=1).reshape(xx.shape)
  else:
    print("Realizando clasificación binaria...")
    y_pred = np.round(np.max(y_pred, axis=1)).reshape(xx.shape)
  
  # Graficar la frontera de decisión
  plt.contourf(xx, yy, y_pred, cmap=plt.cm.RdYlBu, alpha=0.7)
  plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)
  plt.xlim(xx.min(), xx.max())
  plt.ylim(yy.min(), yy.max())

In [ ]:
# Verificación sobre los datos
plot_decision_boundary(modelo_3, X, y)

El modelo intenta dibujar una línea a lo largo de los datos. Sin embargo los datos no son separables por una línea recta. En un problema de regresión, el problema puede funcionar, pero en clasificación no.

## 6. No linealidad

Se puede añadir no linealidad al sistema introduciendo funciones de activación a la salida de las capas densas. Para esto se utilizará el parámetro `activation`.

In [ ]:
tf.random.set_seed(42)

modelo_4 = tf.keras.Sequential([
                                tf.keras.layers.Dense(1, 
                                                      activation = tf.keras.activations.relu), # similar a: activation='relu'
                                tf.keras.layers.Dense(1) # Capa de salida
])

modelo_4.compile(loss = tf.keras.losses.binary_crossentropy,
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics = ["accuracy"])

history = modelo_4.fit(X, y, epochs=100, verbose=0)

In [ ]:
# Evaluación del modelo
modelo_4.evaluate(X, y)

Para mejorar se añadirá capas a la red

In [ ]:
tf.random.set_seed(42)

modelo_5 = tf.keras.Sequential([
                                tf.keras.layers.Dense(4, activation = tf.keras.activations.relu), # capa oculta con 4 neuronas y ReLU
                                tf.keras.layers.Dense(4, activation = tf.keras.activations.relu), # capa oculta con 4 neuronas y ReLU
                                tf.keras.layers.Dense(1) # capa de salida
])

modelo_5.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(lr=0.001), # Por defecto es 0.001
                metrics = ['accuracy'])

history = modelo_5.fit(X, y, epochs=100, verbose=0)

In [ ]:
# Evaluate the model
modelo_5.evaluate(X, y)

Aún se está en 50 % (el modelo está prácticamente adivinando). Es útil visualizar cómo se ven las predicciones

In [ ]:
plot_decision_boundary(modelo_5, X, y)

Se añadirá una función de activación sigmoidea para la capa de salida. De hecho, en clasificación, una salida sigmoidea es usual.

In [ ]:
tf.random.set_seed(42)

modelo_6 = tf.keras.Sequential([
                                tf.keras.layers.Dense(4, activation=tf.keras.activations.relu),
                                tf.keras.layers.Dense(4, activation=tf.keras.activations.relu), 
                                tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid) # Activación sigmoidea
])

modelo_6.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

history = modelo_6.fit(X, y, epochs=100, verbose=0)

In [ ]:
# Evaluate our model
modelo_6.evaluate(X, y)

In [ ]:
plot_decision_boundary(modelo_6, X, y)

## 7. Evaluación y mejora del modelo

Se debe separar los datos en un conjunto de entrenamiento y evaluación (o prueba) y evaluar en el conjunto de evaluación

In [ ]:
# Elementos en el dataset
len(X)

In [ ]:
# Separación de datos
X_train, y_train = X[:800], y[:800] # 80% para entrenamiento
X_test, y_test = X[800:], y[800:]   # 20% para prueba

X_train.shape, X_test.shape 

In [ ]:
tf.random.set_seed(42)

modelo_7 = tf.keras.Sequential([
  tf.keras.layers.Dense(4, activation="relu"), 
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

modelo_7.compile(loss = tf.keras.losses.binary_crossentropy,
                 optimizer = tf.keras.optimizers.Adam(lr=0.01), # Incremento para que sea más rápido
                 metrics = ['accuracy'])

history = modelo_7.fit(X_train, y_train, epochs=25)

In [ ]:
loss, accuracy = modelo_7.evaluate(X_test, y_test)

print(f"Pérdida en el conjunto de prueba: {loss}")
print(f"Exactitud en el conjunto de prueba: {100*accuracy:.2f}%")

Es conveniente una inspección visual


In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Entrenamiento")
plot_decision_boundary(modelo_7, X=X_train, y=y_train)
plt.subplot(1, 2, 2)
plt.title("Prueba")
plot_decision_boundary(modelo_7, X=X_test, y=y_test)
plt.show()

**Curvas de pérdida**

También son llamadas curvas de aprendizaje y muestran cómo se comporta el modelo durante el entrenamiento.

In [ ]:
pd.DataFrame(history.history)

In [ ]:
# Curvas
pd.DataFrame(history.history).plot();
plt.title("Curvas de pérdida para el modelo");

**Factor de aprendizaje**

In [ ]:
tf.random.set_seed(42)

modelo_8 = tf.keras.Sequential([
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

modelo_8.compile(loss = "binary_crossentropy",
                 optimizer = "Adam",
                 metrics = ["accuracy"]) 

# Callback para el factor de aprendizaje
#      Comenzar en 1e-4, e incrementar 10**(epoch/20) cada época
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch/20)) 

history = modelo_8.fit(X_train, 
                      y_train, 
                      epochs = 100,
                      callbacks = [lr_scheduler], 
                      verbose = 0)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10,7), xlabel="epochs");

El factor de aprendizaje se incrementa exponencialmente con el número de épocas. La exactitud se incrementa en un punto específico cuando el factor de aprendizaje se incrementa lento. Se desea encontrar este punto. Para ello, se utilizará un gráfico en escala logarítmica.

In [ ]:
# Factor de aprendizaje vs Pérdida
lrs = 1e-4 * (10 ** (np.arange(100)/20))

plt.figure(figsize=(10, 7))
plt.semilogx(lrs, history.history["loss"]) # Eje x en escala logarítmica
plt.xlabel("Learning Rate")
plt.ylabel("Loss")
plt.title("Learning rate vs. loss");

Un factor de aprendizaje estimado idal es de aproximadamente 0.02. Se utilizará y se re entrenará un modelo.

In [ ]:
tf.random.set_seed(42)

modelo_9 = tf.keras.Sequential([
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

modelo_9.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(lr=0.02), 
                metrics=["accuracy"])

history = modelo_9.fit(X_train, y_train, epochs=20)

Se llega a una exactitud alta con menos épocas (20 en lugar de 25)

In [ ]:
# Evaluar el modelo en el conjunto de prueba
modelo_9.evaluate(X_test, y_test)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Entrenamiento")
plot_decision_boundary(modelo_9, X=X_train, y=y_train)
plt.subplot(1, 2, 2)
plt.title("Prueba")
plot_decision_boundary(modelo_9, X=X_test, y=y_test)
plt.show()

### 8. Otras formas de evaluación

La siguiente tabla muestra algunas de las métricas más usuales

| **Métrica/método** | **Definición** | **Código** |
| --- | --- | --- |
| Exactitud | De cada 100 predicciones, cuántas son correctas | [`sklearn.metrics.accuracy_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) o [`tf.keras.metrics.Accuracy()`](tensorflow.org/api_docs/python/tf/keras/metrics/Accuracy) |
| Precisión | Proporción de verdaderos positivos entre el total de muestras. | [`sklearn.metrics.precision_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html) o [`tf.keras.metrics.Precision()`](tensorflow.org/api_docs/python/tf/keras/metrics/Precision) |
| Recall | Proporción de verdaderos positivos entre el total de verdaderos positivos y falsos negativos (predice 0 cuando es 1). | [`sklearn.metrics.recall_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html) o [`tf.keras.metrics.Recall()`](tensorflow.org/api_docs/python/tf/keras/metrics/Recall) |
| F1-score | Combina precisión y recall. | [`sklearn.metrics.f1_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) |
| [Matriz de confusión | Compara los valores predichos con los reales en formato tabular | [`sklearn.metrics.plot_confusion_matrix()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html) |


In [ ]:
# Exactitud del modelo
loss, accuracy = modelo_9.evaluate(X_test, y_test)

print(f"Pérdida: {loss}")
print(f"Exactitud: {(accuracy*100):.2f}%")

In [ ]:
# Matriz de confusión
from sklearn.metrics import confusion_matrix

# Make predictions
y_preds = modelo_9.predict(X_test)

# Create a confusion matrix
confusion_matrix(y_test, tf.round(y_preds))

In [ ]:
import itertools

figsize = (10, 10)

# Crear la matriz de confusión
cm = confusion_matrix(y_test, tf.round(y_preds))
cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
n_classes = cm.shape[0]

fig, ax = plt.subplots(figsize=figsize)
cax = ax.matshow(cm, cmap=plt.cm.Blues) # https://matplotlib.org/3.2.0/api/_as_gen/matplotlib.axes.Axes.matshow.html
fig.colorbar(cax)

classes = False

if classes:
  labels = classes
else:
  labels = np.arange(cm.shape[0])

ax.set(title="Matriz de Confusión",
       xlabel="Etiquetas predichas",
       ylabel="Etiquetas reales",
       xticks=np.arange(n_classes),
       yticks=np.arange(n_classes),
       xticklabels=labels,
       yticklabels=labels)

ax.xaxis.set_label_position("bottom")
ax.xaxis.tick_bottom()
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
ax.title.set_size(20)
threshold = (cm.max() + cm.min()) / 2.

for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
  plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
           horizontalalignment="center",
           color="white" if cm[i, j] > threshold else "black",
           size=15)